# Compare different batch generation codes

In [1]:
import sys
sys.path.append('../modules/')

%reload_ext autoreload
%autoreload 2
import datasets
import ML_classes

import matplotlib.pyplot as plt

2024-09-05 19:23:24.773138: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 19:23:24.773170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 19:23:24.774448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Old way for making batches

In [10]:
ML_data = datasets.MOM6_all_transformer('-', 'all', 
                                       input_channels = ['U_x', 'U_y', 
                                                         'V_x', 'V_y', 
                                                         'Sx', 'Sy',
                                                        'Lfilt'])

In [11]:
exp = 'DG'
window_size=1
if exp == 'P2L':
    MOM6_bucket = 'gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/'
    
    ML_data.read_datatree(MOM6_bucket, 
                      keep_filt_scale=True, para_perp_out = True)
    
elif exp == 'DG': 
    MOM6_bucket = 'gs://leap-persistent/dhruvbalwada/MOM6/Double_Gyre/res5km/ml_data_'
    ML_data.read_datatree(MOM6_bucket, 
                      file_names='', # need this for file names in DG sim
                      keep_filt_scale=True, 
                      para_perp_out = True, 
                      eta_bottom=True,
                      large_filt=4,
                      H_mask=150,
                      sub_sample=True, 
                      window_size=window_size)
    
L = 'all'
#ML_data = datasets.MOM6_all_transformer('-', 'all')

Dataset loaded from gs://leap-persistent/dhruvbalwada/MOM6/Double_Gyre/res5km/ml_data_50km.zarr
Out para perp
Subsampling
Dataset loaded from gs://leap-persistent/dhruvbalwada/MOM6/Double_Gyre/res5km/ml_data_100km.zarr
Out para perp
Subsampling
Dataset loaded from gs://leap-persistent/dhruvbalwada/MOM6/Double_Gyre/res5km/ml_data_200km.zarr
Out para perp
Subsampling
Dataset loaded from gs://leap-persistent/dhruvbalwada/MOM6/Double_Gyre/res5km/ml_data_400km.zarr
Out para perp
Subsampling


In [12]:
ML_data.datatree = ML_data.datatree.isel(Time=slice(-72, None))

In [14]:
ML_data.generate_test_train_batches(normalize=False)

Test and train batches split. Number of batches: 37-5


/srv/conda/envs/notebook/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [15]:
old_len = len(ML_data.ds_train.points) + len(ML_data.ds_test.points)
old_len

70056

## New way for making batches

In [16]:
DT = datasets.SimulationData(simulation_names=[exp], filter_scales=['50','100','200','400'])
DT.preprocess_simulation_data(window_size=1)

In [17]:
ML_DT_train = datasets.MLDataset(simulation_data=DT,
                                 input_variables=['dudx_widened_rotated_nondim',
                                                  'dvdx_widened_rotated_nondim',
                                                  'dudy_widened_rotated_nondim',
                                                  'dvdy_widened_rotated_nondim',
                                                  'dhdx_widened_rotated_nondim',
                                                  'dhdy_widened_rotated_nondim',
                                                  'filter_scale'],
                                 output_variables=['uphp_rotated_nondim','vphp_rotated_nondim'], 
                                time_range=slice(-72, None))

In [18]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0021 seconds
Time subsampling took: 0.0151 seconds
Horizontal subsampling took: 0.0099 seconds
h_mask_ml_variables took: 0.0791 seconds
stack_physical_dimensions took: 0.0416 seconds
will load upto: 0.0451548 gb
load took: 5.4895 seconds
drop_nans took: 0.0467 seconds
Seed set as:42
randomize_along_points took: 0.0368 seconds
Picked: 80312points
pick_uniform_points took: 0.0036 seconds
concat_datatree_nodes took: 0.0205 seconds
randomize_concatenated_ml_dataset took: 0.0350 seconds
generate_batches took: 0.0002 seconds
CPU times: user 3.35 s, sys: 706 ms, total: 4.05 s
Wall time: 5.78 s


In [19]:
new_len = len(ML_DT_train.concatenated_ml_dataset.points)

In [20]:
diff_len = new_len - old_len
diff_len/old_len

3.585588671919607

The new method has lot more data than the old. Why is this difference there?
- In new method we are using both layers.
- In old method we dropped all data from near boundary, which we are not doing now. 